In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
% matplotlib inline
plt.style.use('ggplot')

import seaborn as sns

import cartopy.crs as ccrs
import cartopy.feature as cfeat
import cartopy.io.img_tiles as cimgt

In [ ]:
def plot_google_map(extent, size = (13, 13)) :
    plt.figure(figsize = size)
    
    img = cimgt.GoogleTiles()

    ax = plt.axes(projection = img.crs)
    ax.set_extent(extent)
    
    ax.add_image(img, 7, interpolation = 'bicubic')

In [ ]:
def plot_osm_map(extent, size = (13, 13)) :
    plt.figure(figsize = size)
    
    img = cimgt.OSM()

    ax = plt.axes(projection = img.crs)
    ax.set_extent(extent)
    
    ax.add_image(img, 7, interpolation = 'bicubic')

In [ ]:
def plot_carto_map(extent, size = (13, 13)) :
    plt.figure(figsize = size)
    
    ax = plt.axes(projection = ccrs.Mercator())
    ax.set_extent(extent)
    
    ax.add_feature(cfeat.LAND.with_scale('50m'))
    ax.add_feature(cfeat.OCEAN.with_scale('50m'))
    ax.add_feature(cfeat.COASTLINE.with_scale('50m'))
    ax.add_feature(cfeat.BORDERS.with_scale('50m'))
    ax.add_feature(cfeat.RIVERS.with_scale('50m'))
    
    # Draw Rotterdam marker
    ax.scatter(x = 4.47917, y = 51.9225, transform = ccrs.PlateCarree(), marker = '*', c = 'C1', s = 500)
    # Draw Hamburg marker
    ax.scatter(x = 10.01534, y = 53.57532, transform = ccrs.PlateCarree(), marker = '*', c = 'C1', s = 500)

In [ ]:
def plot_to_map(size, longitude, latitude) :
    min_long = longitude.min() - 0.5
    max_long = longitude.max() + 0.5
    min_lat = latitude.min() - 0.5
    max_lat = latitude.max() + 0.5
    
    plot_carto_map((min_long, max_long, min_lat, max_lat), size = size)
    
    plt.scatter(x = longitude, y = latitude, transform = ccrs.PlateCarree(), s = 1)

In [ ]:
# Load the data
names = ['TripID', 'MMSI', 'StartLatitude', 'StartLongitude', 'StartTime', 'EndLatitude', 'EndLongitude', 'EndTime',
         'StartPort', 'EndPort', 'ID', 'time', 'shiptype', 'Length', 'Breadth', 'Draught', 'Latitude', 'Longitude',
         'SOG', 'COG', 'TH', 'Destination', 'Name', 'Callsign', 'AisSourcen']
ais = pd.read_csv('rotterdam_hamburg.arff', names = names, skiprows = 27, parse_dates = True, index_col = 'ID',
                  na_values = ['?'], dtype = {'TripID': str, 'MMSI': str, 'shiptype': str})

In [ ]:
ais.head()

In [ ]:
ais_nan = ais

In [ ]:
# Convert time columns to correct dtype
ais_nan['StartTime'] = pd.to_datetime(ais_nan['StartTime'], format = '\'%Y-%m-%d %H:%M\'')
ais_nan['EndTime'] = pd.to_datetime(ais_nan['EndTime'], format = '\'%Y-%m-%d %H:%M\'')
ais_nan['time'] = pd.to_datetime(ais_nan['time'], format = '\'%Y-%m-%d %H:%M\'')

In [ ]:
# Convert all headings that are 511 (>= 360) to NaN
ais_nan.loc[ais_nan['TH'] >= 360, 'TH'] = np.nan

In [ ]:
# Convert courses >= 360 to NaN
ais_nan.loc[ais_nan['COG'] >= 360, 'COG'] = np.nan

In [ ]:
# Set invalid shiptypes to NaN
# Invalid shiptypes existing in the data set are '0' and '159'
ais_nan.loc[(ais_nan['shiptype'] == '0') | (ais_nan['shiptype'] == '159'), 'shiptype'] = np.nan

In [ ]:
# Set invalid lengths (0 or > 400) to NaN
ais_nan.loc[(ais_nan['Length'] <= 0) | (ais_nan['Length'] > 400), 'Length'] = np.nan

In [ ]:
# Set speeds that are unrealisticly high to NaN
ais_nan.loc[ais_nan['SOG'] > 25.6, 'SOG'] = np.nan

In [ ]:
ais_dropped = ais_nan.drop(['AisSourcen', 'StartPort', 'EndPort', 'Name', 'MMSI', 'shiptype'], axis = 1)

In [ ]:
ais_dropped.head()

In [ ]:
# Plot all positions in the data before any filters are applied
plot_to_map(size = (13, 13), longitude = ais_dropped['Longitude'], latitude = ais_dropped['Latitude'])
plt.title("All Positions Before Filtering")

In [ ]:
# Plot start positions before any filtering to map
plot_to_map(size = (8, 8), longitude = ais_dropped['StartLongitude'], latitude = ais_dropped['StartLatitude'])
plt.title("Start Positions Before Filtering")

In [ ]:
# Plot end positions before any filtering to map
plot_to_map(size = (8, 8), longitude = ais_dropped['EndLongitude'], latitude = ais_dropped['EndLatitude'])
plt.title("End Positions Before Filtering")

In [ ]:
# Get rid of trips that leave our zone (2.45 - 10.66 / 51.49 - 55.06)

outside_long_mask = (ais_dropped['Longitude'] < 2.45) | (ais_dropped['Longitude'] > 10.66)
outside_lat_mask = (ais_dropped['Latitude'] < 51.49) | (ais_dropped['Latitude'] > 55.06)

outside_rec_mask = outside_long_mask | outside_lat_mask

outside_trip_ids = ais_dropped['TripID'][outside_rec_mask].unique()
outside_trip_mask = ~ais_dropped['TripID'].isin(outside_trip_ids)
ais_zoned = ais_dropped[outside_trip_mask]

In [ ]:
plot_to_map(size = (13, 13), longitude = ais_zoned['Longitude'], latitude = ais_zoned['Latitude'])
plt.title("All Positions After Filtering to Our Zone")

In [ ]:
ais_tts = ais_zoned

# Add time for the entire trip to the data
ais_tts['TravelTime'] = ais_tts['EndTime'] - ais_tts['StartTime']
ais_tts['TravelTimeMins'] = ais_tts['TravelTime'].transform(lambda x : x.total_seconds() / 60)

# Add time remaining until arrival at destination
ais_tts['remainingTT'] = ais_tts['EndTime'] - ais_tts['time']
ais_tts['remainingMins'] = ais_tts['remainingTT'].transform(lambda x : x.total_seconds() / 60)

In [ ]:
travel_times = ais_tts.groupby('TripID')['TravelTime'].max()
travel_times = travel_times.transform(lambda x : x.total_seconds() // 3600)
travel_times.value_counts().sort_index().plot.bar(figsize = (25, 10))

In [ ]:
travel_times[travel_times < travel_times.quantile(.75)].value_counts().sort_index().plot.bar(figsize = (25, 10))

In [ ]:
# Only keep majority of shorter trips
ais_shortmaj = ais_tts[ais_tts['TravelTime'] < ais_tts['TravelTime'].quantile(.75)]
ais_shortmaj['TravelTime'].describe()

In [ ]:
plot_to_map(size = (13, 13), longitude = ais_shortmaj['Longitude'], latitude = ais_shortmaj['Latitude'])

In [ ]:
# Get and plot the longest trip in the data set
longest_trip = ais_shortmaj[ais_shortmaj['TravelTime'] == ais_shortmaj['TravelTime'].max()]
longest_trip.head()

In [ ]:
plot_to_map(size = (13, 13), longitude = longest_trip['Longitude'], latitude = longest_trip['Latitude'])
plt.title("Longest Trip in the Data Set")

In [ ]:
from math import sin, cos, sqrt, atan2, radians

def dist(a_lat, a_long, b_lat, b_long) :
    R = 6373.0
    
    lat1 = radians(a_lat)
    lon1 = radians(a_long)
    lat2 = radians(b_lat)
    lon2 = radians(b_long)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [ ]:
sns.pairplot(ais_shortmaj, x_vars = ['Latitude', 'Longitude'], y_vars = 'remainingMins', size = 7)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics

In [ ]:
X = ais_shortmaj[['Latitude', 'Longitude']]
y = ais_shortmaj['remainingTT']
y = y.transform(lambda x : x.total_seconds() / 60)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .1)

In [ ]:
lin = LinearRegression()
lin.fit(X_train, y_train)

In [ ]:
lin_pred = lin.predict(X_test)
sqrt(metrics.mean_squared_error(y_test, lin_pred))

In [ ]:
knn = KNeighborsRegressor(n_neighbors = 100)
knn.fit(X_train, y_train)

In [ ]:
knn_pred = knn.predict(X_test)
sqrt(metrics.mean_squared_error(y_test, knn_pred))

In [ ]:
import math

k_range = range(1, 30)
scores = []

for k in k_range :
    kknn = KNeighborsRegressor(n_neighbors = k)
    kknn.fit(X_train, y_train)
    kknn_pred = kknn.predict(X_test)
    scores.append(sqrt(metrics.mean_squared_error(y_test, kknn_pred)))

In [ ]:
plt.figure(figsize = (15, 5))
plt.plot(k_range, scores)